In [2]:
import pickle

import models
import em

# Inductor GR 0.35um 


Octagonal inductor in AMS 350 um.


In [3]:
with open('../RBF_inductor_5G_SRF7_200GHz_tmtt.model','rb') as infile:
    model = pickle.load( infile)

In [5]:
model['device'] = model.pop('model')
model['ranges'] = {'din': (70, 180), 'w': (6, 14), 'nt': (1,5), 'freq': (0, 200)}

In [7]:
print(model['inputs'])
print(model['key'])

['Din', 'W']
{'nt': [1, 2, 3, 4, 5], 'f': (0, 203)}


In [8]:
with open("../PASSIVES_RBF_IND_5G_SRF7_200GHz_tmtt.model",'wb') as outfile:
    pickle.dump(model, outfile)  


In [10]:
with open('../RBF_transf_28G_SRF38.model','rb') as infile:
    trans = pickle.load( infile)

In [14]:
trans['device'] = 'transf'
trans['ranges'] = {'dinp': (30, 190), 'wp': (4, 14), 'np': (1,1), 'dins': (30, 190), 'ws': (4, 14), 'ns': (1,1), 'freq': (0, 200)}

In [11]:
print(trans['inputs'])
print(trans['key'])

['Dinp', 'Wp', 'Dins', 'Ws']
{'nt': [(1, 1, '_balun'), (1, 1, '')], 'f': (0, 201)}


In [15]:
with open("../PASSIVES_RBF_TRANSF_28G_SRF38.model",'wb') as outfile:
    pickle.dump(trans, outfile)  

In [18]:
with open('../RBF_transf_77G_SRF87.model','rb') as infile:
    trans = pickle.load( infile)

In [19]:
print(trans['inputs'])
print(trans['key'])

['Dinp', 'Wp', 'Dins', 'Ws']
{'nt': ((1, 1), (2, 2)), 'f': (0, 201)}


In [14]:
#turn and freq independent @ 5GHz SRF > 7GHz
import pickle
import em
import importlib

importlib.reload(models)

np.random.seed(1234)
tf.random.set_seed(1234)

model_table = {}
for nt in range(1, 6):
    for f in range(204):
        freq, x_train, y_train, x_test, y_test, srf_data = models.load_data(data_folder, f, nt = nt, srf=7.0, skip_test=True)
        m = models.RBFInterpolatorModel(degree=4)
        m.fit(x_train, y_train)
        mdl = {
            'freq_index': f,
            'freq': freq[0],
            'nt': nt,
            'RBFmodel': m
        }

        model_table[(nt, f)] = mdl
    
    srf_m = models.RBFInterpolatorModel(degree=2)
    srf_m.fit(srf_data[0], srf_data[1])        
    srf_mdl = {
            'srf_limit': 7,
            'nt': nt,
            'RBFmodel': srf_m
    }
    model_table[(nt, 'srf')] = srf_mdl


model_table['inputs'] = ['Din', 'W']
model_table['ouputs'] = em.COLUMNS_2P[1:]
model_table['model'] = 'ind'
model_table['key'] = {'nt': [1, 2, 3, 4 ,5], 'f':(0, 203)}

with open("../RBF_inductor_5G_SRF7_200GHz_tmtt.model",'wb') as outfile:
    pickle.dump(model_table, outfile)    


In [ ]:
def create_model(fname, data_folder):
    model_table = {}
    for nt in range(1, 6):
        for f in range(204):
            freq, x_train, y_train, x_test, y_test, srf_data = models.load_data(data_folder, f, nt = nt, srf=7.0, skip_test=True)
            m = models.RBFInterpolatorModel(degree=4)
            m.fit(x_train, y_train)
            mdl = {
                'freq_index': f,
                'freq': freq[0],
                'nt': nt,
                'RBFmodel': m
            }

            model_table[(nt, f)] = mdl
        
        srf_m = models.RBFInterpolatorModel(degree=2)
        srf_m.fit(srf_data[0], srf_data[1])        
        srf_mdl = {
                'srf_limit': 7,
                'nt': nt,
                'RBFmodel': srf_m
        }
        model_table[(nt, 'srf')] = srf_mdl


    model_table['inputs'] = ['Din', 'W']
    model_table['ouputs'] = em.COLUMNS_2P[1:]
    model_table['model'] = 'ind'
    model_table['key'] = {'nt': [1, 2, 3, 4 ,5], 'f':(0, 203)}

    with open(fname,'wb') as outfile:
        pickle.dump(model_table, outfile) 

In [ ]:
create_model('test_model.model', '../data/inductGR_350nm_200GHz/')